In [48]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import pickle
import random
import plotly.express as px

In [3]:
from nxviz import CircosPlot

In [4]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
file = 'rete.pickle'
with open(path + file, 'rb') as f:
    G = pickle.load(f)

In [5]:
#Numero di nodi
N = G.number_of_nodes()
N

22

In [6]:
#Numero di archi
L = G.number_of_edges()
L

123

In [7]:
#densità
d = (2*L)/(N*(N-1))
d

0.5324675324675324

In [8]:
#Grado medio
k_avg = 2*L/N
k_avg

11.181818181818182

In [9]:
#Gradi dei nodi
gradi_nodi_dict = dict(G.degree())

In [10]:
#Grado massimo minimo
print( min(list(gradi_nodi_dict.values())) , max(list(gradi_nodi_dict.values())) )

0 17


In [11]:
#Assortatività
gradi_origine = []
gradi_destinazione = []
for u, v in G.edges():
    gradi_origine.append(gradi_nodi_dict[u])
    gradi_destinazione.append(gradi_nodi_dict[v])
coeff_pearson = np.corrcoef(gradi_origine, gradi_destinazione)[0, 1]
coeff_pearson

-0.19417264635430426

In [12]:
#Componeneti connesse
componenti_connesse = list(nx.connected_components(G))
max([len(x) for x in componenti_connesse])

21

In [13]:
#Diametro
nx.diameter(G.subgraph(componenti_connesse[0]))

3

In [14]:
clustering = nx.average_clustering(G)
clustering

0.8017472574959205

In [15]:
avg_shortest_path = nx.average_shortest_path_length(G.subgraph(componenti_connesse[0]))
avg_shortest_path

1.438095238095238

In [16]:
degree_distribution = {}
for degree in gradi_nodi_dict.values():
    if degree in degree_distribution:
        degree_distribution[degree] += 1
    else:
        degree_distribution[degree] = 1

In [17]:
sorted_degree_distribution = sorted(degree_distribution.items(), key=lambda x: x[1], reverse=True)

In [18]:
degrees, frequencies = zip(*sorted_degree_distribution)

In [19]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=degrees,
    x=frequencies,
    name='AVG polarity per week',
    orientation='h',
    text=list(frequencies),
    textposition='outside', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.update_layout(
    title=dict(
        text='<b>Distribuzione di frequenza del grado dei nodi',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
        ),
    yaxis=dict(
    title='Gradi',
    tickmode='array',
    tickvals=list(range(min(degrees),max(degrees)+1)),  # Posizioni delle etichette sull'asse Y
    ticktext=list(range(min(degrees),max(degrees)+1)),  # Testo delle etichette sull'asse Y
    ),
    xaxis=dict(
         title='Frequenze'
    )
)
fig.update_layout(
    width=1000,   # Larghezza in pixel
    height=600   # Altezza in pixel
)

In [20]:
for v in G:
    G.nodes[v]["class"] = G.degree(v)

In [21]:
for v in G:
    G.nodes[v]["dim"] = G.nodes[v]["size"]/12

In [22]:
G.nodes(data=True)

NodeDataView({'AI Chatbot Interaction and Assistance on Cloud Platforms': {'size': 12.415028018989009, 'class': 15, 'dim': 1.0345856682490842}, 'Chatbot Interaction and Usage on Discord': {'size': 13.857746911806666, 'class': 11, 'dim': 1.1548122426505556}, 'Dan Discussing ChatGPT Jailbreaking Ethics': {'size': 13.251238446159192, 'class': 0, 'dim': 1.104269870513266}, 'AI and ChatGPT in Everyday Life': {'size': 16.094182292170824, 'class': 5, 'dim': 1.341181857680902}, 'Reddit Karma and Account Restrictions': {'size': 10.995729594196394, 'class': 9, 'dim': 0.9163107995163662}, 'Social Media Karma and Automated Account Moderation': {'size': 11.20581589485235, 'class': 13, 'dim': 0.9338179912376958}, 'ChatGPT Use Cases and Discussions': {'size': 9.721485944553093, 'class': 7, 'dim': 0.8101238287127578}, 'Obama and Trump References in AI Conversations': {'size': 10.286843375056192, 'class': 3, 'dim': 0.8572369479213493}, 'AI Chatbot Interaction and Response automation': {'size': 13.03550

In [23]:
G.nodes['Dan Discussing ChatGPT Jailbreaking Ethics']['class'] = 2

In [25]:
giant_cc = G.subgraph(componenti_connesse[0])

In [26]:
shotest_paths_u_v = []
visited_pairs = set()
for nodo1 in giant_cc.nodes():
    for nodo2 in giant_cc.nodes():
        if nodo1 != nodo2 and (nodo1, nodo2) not in visited_pairs and (nodo2, nodo1) not in visited_pairs:
            shotest_paths_u_v.append(nx.shortest_path_length(G, source=nodo1, target=nodo2))
            visited_pairs.add((nodo1, nodo2))

In [27]:
frequenze_s = {}
for sp in shotest_paths_u_v:
    if sp in frequenze_s:
        frequenze_s[sp] += 1
    else:
        frequenze_s[sp] = 1

In [124]:
frequenze_s.keys()

dict_keys([1, 2, 3])

In [128]:


fig = go.Figure()
fig.add_trace(go.Bar(
    x=list(frequenze_s.keys()),
    y=[f/len(shotest_paths_u_v)  for f in frequenze_s.values()],
    name='Distribuzione delle distanze minime',
    orientation='v',
    text=list(frequencies),
    textposition='outside', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.update_layout(
    title=dict(
        text='<b>Distribuzione delle distanze minime',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
        ),
    xaxis=dict(
    title='Distanza minima'
    ),
    yaxis=dict(
         title='Frequenza'
    )
)
fig.update_layout(
    width=500,   # Larghezza in pixel
    height=600   # Altezza in pixel
)

In [122]:
fig = go.Figure()
colors = ['rgb(255, 0, 0)',   # Rosso
          'rgb(0, 255, 0)',   # Verde
          'rgb(0, 0, 255)',   # Blu
          'rgb(255, 255, 0)', # Giallo
          'rgb(255, 0, 255)', # Magenta
          'rgb(0, 255, 255)', # Ciano
          'rgb(255, 128, 0)', # Arancione
          'rgb(128, 0, 255)', # Viola
          'rgb(0, 255, 128)',
          'rgb(128, 128, 128)', # Grigio # Verde acqua
          'rgb(255, 0, 128)'  # Rosa
         ]
for j in range(1,12):
    frac_node_gcc = []
    frac_nodes_removed = []
    G1 = G.copy()
    gcc = giant_cc.copy()
    lista_nodi = list(G1.nodes())
    frac_nodes_removed.append(round((0/N),2))
    frac_node_gcc.append(round((len(gcc)/N),2))
    for i in range(1,len(lista_nodi)):
        if j<=10:
            index_to_remove = random.randrange(0,len(lista_nodi))
            nodo_da_rimuovere = lista_nodi.pop(index_to_remove)
        else:
            degree_dict = dict(G1.degree())
            nodo_da_rimuovere = max(degree_dict, key=degree_dict.get)
        G1.remove_node(nodo_da_rimuovere)
        lista_nodi = list(G1.nodes())
        ccs = nx.connected_components(G1)
        gcc = max(ccs,key=len)
        frac_nodes_removed.append(round((i/N),2))
        frac_node_gcc.append(round((len(gcc)/N),2))
    frac_nodes_removed.append(round(1,2))
    frac_node_gcc.append(round(0,2))
    if j<=10:
        fig.add_trace(go.Scatter(x=frac_nodes_removed, y=frac_node_gcc, mode='lines+markers', line=dict(color=colors[j-1]), name=f'random attack {j}'))
    else:
        fig.add_trace(go.Scatter(x=frac_nodes_removed, y=frac_node_gcc, mode='lines+markers', line=dict(color=colors[j-1]), name=f'targeted attack'))
fig.update_layout(    title=dict(
    text='<b>Network robustness',  # Title text
    x=0.5,  # x=0.5 centers the title
    xanchor='center',  # 'center' ensures the title is centered
    font=dict(
        size=24,
        color='black',
        family="Arial, sans-serif",
    )
    ),
                xaxis_title='Proportion of nodes removed',
                yaxis_title='Proportion of nodes in giant component',
                width=800,   # Larghezza in pixel
                height=600 )

fig.show()


In [120]:
fig = go.Figure()
frac_node_gcc = []
frac_nodes_removed = []
G1 = G.copy()
gcc = giant_cc.copy()
lista_nodi = list(G1.nodes())
frac_nodes_removed.append(round((0/N),2))
frac_node_gcc.append(round((len(gcc)/N),2))
for i in range(1,len(lista_nodi)):
    degree_dict = dict(G1.degree())
    nodo_da_rimuovere = max(degree_dict, key=degree_dict.get)
    G1.remove_node(nodo_da_rimuovere)
    lista_nodi = list(G1.nodes())
    ccs = nx.connected_components(G1)
    gcc = max(ccs,key=len)
    frac_nodes_removed.append(round((i/N),2))
    frac_node_gcc.append(round((len(gcc)/N),2))
frac_nodes_removed.append(round(1,2))
frac_node_gcc.append(round(0,2))
fig.add_trace(go.Scatter(x=frac_nodes_removed, y=frac_node_gcc, mode='lines+markers', line=dict(color=colors[-1])))
fig.update_layout(    title=dict(
    text='<b>Network robustness targeted attacks',  # Title text
    x=0.5,  # x=0.5 centers the title
    xanchor='center',  # 'center' ensures the title is centered
    font=dict(
        size=24,
        color='black',
        family="Arial, sans-serif",
    )
    ),
                xaxis_title='Proportion of nodes removed',
                yaxis_title='Proportion of nodes in giant component',
                width=800,   # Larghezza in pixel
                height=600 )

fig.show()